# Защита персональных данных клиентов

Нам необходимо защитить данные клиентов страховой компании.

Перед нами поставлена задача по разработке такого метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию.

Необходимо защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

# План работы
-Загрузка данных

-Умножение матриц

-Алгоритм преобразования

-Проверка алгоритма

-Вывод

##  Загрузка данных

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [ ]:
try:
    df = pd.read_csv('/datasets/insurance.csv')
except:
    df = pd.read_csv('C:/Users/www//insurance.csv')

In [ ]:
df

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


In [ ]:
full = [df]

for dataset in full:
    print('\n')
    print('Колличество строк, столбцов', dataset.shape)
    print('Количество пропусков:', dataset.isna().sum().sum())
    print('Количество дубликатов:', dataset.duplicated().sum())



Колличество строк, столбцов (5000, 5)
Количество пропусков: 0
Количество дубликатов: 153


In [ ]:
df.isna().mean().sort_values()

Пол                  0.0
Возраст              0.0
Зарплата             0.0
Члены семьи          0.0
Страховые выплаты    0.0
dtype: float64

In [ ]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [ ]:
df.corr()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
Пол,1.000000,0.002074,0.014910,-0.008991,0.010140
Возраст,0.002074,1.000000,-0.019093,-0.006692,0.651030
Зарплата,0.014910,-0.019093,1.000000,-0.030296,-0.014963
Члены семьи,-0.008991,-0.006692,-0.030296,1.000000,-0.036290
Страховые выплаты,0.010140,0.651030,-0.014963,-0.036290,1.000000


##  Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** R2 метрика не измениться

**Обоснование:**
Значит:
    
$$w' = ((XP)^T XP)^{-1} (XP)^T y$$
    
    
Вот так мы сделать не можем:    
    
$((XР)^T(XР))^{-1} = (XP)^{-1}((XP)^T)^{-1}$    

    
Потому что произведение

    
$XP$    

не квадратная матрица, следовательно
    
$(XP)^{-1}$и $((XP)^{T})^{-1} $  и $X^{-1}$      
    
не существуют, это как на ноль делить.
    
    
раскроем под знаком транспонирования:
    
$$w' = (P^T X^T X P)^{-1} (XP)^T y$$
    
А теперь поставим скобочки чтобы лучше было видно:
    
    
$$w' = (P^T (X^T X) P)^{-1} P^T X^T y$$    
    

и видим 3 квадратные матрицы!
    
$P^T, (X^T X), P$   
    
Используем правило:
    
    
$$(ABC)^{-1} = C^{-1}(AB)^{-1} = (BC)^{-1}A^{-1} = C^{-1}B^{-1}A^{-1}$$

$$(P^T(X^TX)P)^{-1} = P^{-1}(X^TX)^{-1}(P^T)^{-1} = P^{-1}(X^TX)^{-1}P^{-T}$$

Теперь можем заменить в исходном выражении:

$$w' = (P^{-1}(X^TX)^{-1}P^{-T})P^TX^Ty = P^{-1}(X^TX)^{-1}(P^{-T}P)X^Ty = P^{-1}(X^TX)^{-1}X^Ty$$


Таким образом, окончательное выражение для $w'$ выглядит следующим образом:

$$w' = P^{-1}(X^TX)^{-1}X^Ty$$   

Наша формула обучения:

$$ w = (X^T X)^{-1} X^T y $$


Теперь выводим соотношение  $a = a_p$, то $XPa = Xw$.

Переписывая это выражение через векторы весов $w$ и $w_p$, получим:

$$XPw_p = Xw$$

То есть векторы $w$ и $w_p$ будут удовлетворять этому соотношению, если $w_p = P^{-1}w$.

Можно проверить это, используя определение матрично-векторного произведения:

$$X(P^{-1}w) = (XP^{-1})w = Xw$$

Таким образом, при $w_p = P^{-1}w$, уравнения $Xw_p = Xw$ будут выполняться.


In [ ]:
# Разделим данные
X = df.drop("Страховые выплаты", axis=1)
y = df["Страховые выплаты"]
X.shape, y .shape

((5000, 4), (5000,))

In [ ]:
# Создам матрицу, предположим что она обратима
INVERT = np.array([
    [1, 27, 55, 122],
    [2, 31, 65, 113],
    [1, 19, 46, 98],
    [3, 34, 87, 130]])

# Проверим, обратима ли матрица
np.linalg.inv(INVERT)

array([[-6.19343946,  5.0525808 ,  5.68596237, -2.86589484],
       [ 0.09406657,  0.12156295, -0.19392185, -0.04775687],
       [ 0.36710082, -0.36661843, -0.35166425,  0.23926676],
       [-0.12735166,  0.09696093,  0.15484805, -0.07380608]])

матрица обратима, т.к. не обнаружена ошибка.

In [ ]:
def predict_r2_score(X, y):
    lr = LinearRegression()
    lr.fit(X, y)
    r2 = lr.score(X, y)
    return r2

In [ ]:
# r2 для обычной матрицы признаков
predict_r2_score(X, y)

0.4249455028666801

In [ ]:
# r2 для случайной обратной матрицы
X_invert = X @ INVERT
predict_r2_score(X_invert, y)

0.4249455028667417

Качество линейнеой регрессии, при умножении матрицы признаков nxk справа на квадратную обратимую матрицу kxk, не изменилось, метрика R2 одинакова

##  Алгоритм преобразования

Для защиты информации на этапе поиска коэффициентов регрессии и предсказаний регрессии будем умножать матрицу признаков на обратимую матрицу
, которая была сгенерирована случайным образом.

##  Проверка алгоритма

In [ ]:
def get_rand_matrix():
    det = 0
    while det == 0:
        matrix = np.random.normal(size=(data.shape[1], data.shape[1]))
        det = np.linalg.det(matrix)
    return matrix

In [ ]:
def func(dim):
    transform_matrix = np.random.rand(dim, dim)
    try:
        inv = np.linalg.inv(transform_matrix)
        return transform_matrix, inv
    except:
        func(dim)

In [ ]:
get_rand_matrix

<function __main__.get_rand_matrix()>

In [ ]:
func

<function __main__.func(dim)>

In [ ]:
X_new = ((X*10000 - 100) * 5) @ INVERT

X_new.head()

predict_r2_score(X_new, y)

0.42494550286662713

In [ ]:
X_new = ((X*10000 - 100) * 15) @ INVERT

X_new.head()

predict_r2_score(X_new, y)

0.42494550286673416

# Вывод:
Нам была поставлена задача по защите данных клиентов страховой компании «Хоть потоп».

Нам удалось разработать метод преобразования данных, основанный на получении случайно матрицы и умножению ее на матрицу обучающих признаков. Это позволит исказить данные визуально, но при этом данные преобразования не влияют на эффективность работы модели машинного обучения - качество предсказаний не меняет.